In [1]:
# %config Completer.use_jedi = False
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

**read raw data**

In [2]:
path = './data1/'
raw = pd.read_csv(path+'train.csv')
print(raw['customer_id'].nunique())

1585986


**数据处理**

In [3]:
# 处理日期
time_columns = ['order_pay_time']
for col in time_columns:
    raw[col] = pd.to_datetime(raw[col], format="%Y-%m-%d")
# 1月之前的数据都去掉
raw = raw[raw['order_pay_time']>dt.datetime(2013, 1, 1)]    
print(raw['customer_id'].nunique())
# 处理类别特征
raw['customer_gender'], _ = pd.factorize(raw['customer_gender'], na_sentinel=0)
raw['customer_province'], _ = pd.factorize(raw['customer_province'], na_sentinel=-1)
raw['customer_city'], _ = pd.factorize(raw['customer_city'], na_sentinel=-1)
# raw['goods_id'], _ = pd.factorize(raw['goods_id'], na_sentinel=-1)

1063573


In [4]:
print(raw['customer_id'][(raw['order_pay_time']>dt.datetime(2013, 8, 1))&(raw['order_pay_time']<dt.datetime(2013, 9, 1))].nunique())

173385


**准备数据**

In [16]:
# 返回某段时间内的统计数据
'''
Parameters：
----------
timestamp: 从该时间往前统计
''' 
def time_window_aggs(timestamp, is_train=True):
    window_lengths = [14, 30, 60, 91]
    # 准备这段时间内的原始数据
    start_time = timestamp-dt.timedelta(max(window_lengths))
    end_time = timestamp.replace(month=timestamp.month+1)
    data = raw[(raw['order_pay_time']>start_time)&(raw['order_pay_time']<end_time)]
    data = data.sort_values(['customer_id', 'order_pay_time'])
    
    data_features = data[data['order_pay_time']<timestamp]
    # 初始化aggs
    aggs = pd.DataFrame()
    aggs['customer_id'] = data_features['customer_id'].unique()
    
    """label""" 
    if is_train:
        data_label = data[data['order_pay_time']>timestamp]
        df = data_label.groupby('customer_id')
        tmp = df.agg({'order_total_payment':'max'})
        tmp['order_total_payment'][tmp['order_total_payment']>0] = 1
        aggs = merge_rename_astype(aggs, tmp, 'order_total_payment', 'is_buyed', 0, np.uint8)
        print(aggs['is_buyed'].sum()/len(aggs))
      
    """特征"""
    print(data_features['order_pay_time'].min(), data_features['order_pay_time'].max())
    # 不同时间窗统计量
    for window_length in window_lengths:
        # 时间窗开始时间
        start_time = timestamp-dt.timedelta(window_length)
        # 获取时间窗口内数据并groupby “customer_id"
        df = data_features[data_features['order_pay_time']>start_time].groupby('customer_id')

        """订单部分"""
        # 这里都选取order_detail的数据，order_detail对应子订单，一个order可以包含多个order_detail，
        # 这里认为子订单更能反应订单特性

        # 统计选取特征的sum, max
        cols = [ 'order_detail_amount', 'order_detail_goods_num','order_detail_payment']
        for col in cols:
            tmp = df.agg({col:'sum'})
            # merge, rename, fillna and change type for reduce memory
            aggs = merge_rename_astype(aggs, tmp, col, f'{window_length}_Sum_'+col, 0, np.float16)
            
            tmp = df.agg({col:'max'})
            # merge, rename, fillna and change type for reduce memory
            aggs = merge_rename_astype(aggs, tmp, col, f'{window_length}_Max_'+col, 0, np.float16)
        # 统计订单数量
        tmp = df['order_detail_id'].count()
        aggs = merge_rename_astype(aggs, tmp, 'order_detail_id', f'{window_length}_Count_order', 0, np.uint16)
    
    # 最大范围时间统计量
    df = data_features.groupby('customer_id')
    """用户相关"""
    # 用户是否评价
    col = 'is_customer_rate'
    tmp = df.agg({col:'mean'})
    aggs = merge_rename_astype(aggs, tmp, col, 'Mean_'+col, 0, np.float16)
    # 最后一次订单发生时用户的会员状态, member_status数据和is_member_actived完全一样，只取一个
    col = 'is_member_actived'
    tmp = df[col].last()
    aggs = merge_rename_astype(aggs, tmp, col, 'Last_'+col, 0, np.uint8) 

    """商品相关"""
    # 购买商品种类  （数据中goods_id和goods_class_id完全相同）
    tmp = df['goods_id'].nunique()
    aggs = merge_rename_astype(aggs, tmp, 'goods_id', 'Count_goods', 0, np.uint16)
    # 是否有折扣
    col = 'goods_has_discount'
    tmp = df.agg({col: 'sum'})
    aggs = merge_rename_astype(aggs, tmp, col, 'Sum_'+col, 0, np.uint16)
    # 商品种类作为类别特征
    return aggs

def cat_aggs(aggs):
    df = raw.groupby('customer_id')
    # 性别
    tmp = df['customer_gender'].last()
    aggs = merge_rename_astype(aggs, tmp, 'customer_gender', 'customer_gender', 0, "category")
    # 省份
    tmp = df['customer_province'].last()
    aggs = merge_rename_astype(aggs, tmp, 'customer_province', 'customer_province', -1, "category")
    # 城市
    tmp = df['customer_city'].last()
    aggs = merge_rename_astype(aggs, tmp, 'customer_city', 'customer_city', -1, "category")
    return aggs

def merge_rename_astype(aggs, df, column_name, new_column_name, fillna_value, dtype):
    aggs = aggs.merge(df, how='left', on='customer_id')
    aggs.rename(columns={column_name: new_column_name}, inplace=True)
    aggs[new_column_name] = aggs[new_column_name].fillna(fillna_value).astype(dtype)
    return aggs


In [17]:
# 训练数据集时间节点
# train_times = [dt.datetime(2013, m, 1) for m in range(5, 9)]
train_times = [dt.datetime(2013, 7, d) for d in range(7, 31, 7)]
# 构建数据
train_data = []
for timestamp in train_times:
    data_one_time = time_window_aggs(timestamp, is_train=True)
    data_one_time = cat_aggs(data_one_time)  
    train_data.append(data_one_time)
# 形成完整训练集
train_data = pd.concat(train_data)


0.088428344833876
2013-04-07 00:00:13 2013-07-06 23:59:31
0.11701124417411875
2013-04-14 00:00:33 2013-07-13 23:59:52
0.14918863115178743
2013-04-21 00:00:17 2013-07-20 23:59:37
0.15434859666426282
2013-04-28 00:00:11 2013-07-27 23:59:41


In [18]:
# 测试数据集时间节点
test_time = dt.datetime(2013, 9, 1)
# 构建数据
test_data = time_window_aggs(test_time, is_train=False)
test_data = cat_aggs(test_data)

2013-06-02 00:00:03 2013-08-31 23:59:59


In [19]:
X = train_data.drop(['customer_id', 'is_buyed'], axis=1)
y = train_data['is_buyed']
X_predict = test_data.drop(['customer_id'], axis=1)

In [20]:
X_total = pd.concat([X, X_predict])

In [21]:
X_total.drop(columns=['customer_city', 'customer_province'], inplace=True, axis=1)

In [22]:
# X_total = pd.get_dummies(X_total, columns=['customer_gender', 'customer_province'])
X_total = pd.get_dummies(X_total, columns=['customer_gender'])

In [23]:
X = X_total[:len(X)]
X_predict = X_total[len(X):]

In [24]:
X.to_pickle('./data1/X_train.pkl')
y.to_pickle('./data1/y_train.pkl')
X_predict.to_pickle('./data1/X_test.pkl')
test_data['customer_id'].to_pickle('./data1/y_test.pkl')